In [1]:
import os 
%pwd
os.chdir('../')

In [2]:
%pwd

'e:\\ML_Projects_iNeuron\\End_end_Eye_Disease_detection_using_Mlops_Project'

In [12]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/shreyasmendhekar77/End_to_End_Eyes_Cataract_detection_using_Deep_learning.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="shreyasmendhekar77"
os.environ["MLFLOW_TRACKING_PASSWORD"]="40a68921fc18c53ca9b654767e357ed1bc9e3903"

In [13]:
import tensorflow as tf
# model = tf.keras.models.load_model("artifacts/training/model.h5")

In [29]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    testing_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int


In [30]:
from cnnClassifier_Eye.constants import *
from cnnClassifier_Eye.utils.common import read_yaml, create_directories, save_json


In [42]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        print(self.config.training.test_data_path)

        create_directories([self.config.artifacts_root])

    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            testing_data=self.config.training.test_data_path,
            mlflow_uri="https://dagshub.com/shreyasmendhekar77/End_to_End_Eyes_Cataract_detection_using_Deep_learning.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        
        return eval_config

In [43]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse


In [44]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.testing_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = self.model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="ResNet50Model")
            else:
                mlflow.keras.log_model(self.model, "model")


In [45]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2025-05-09 21:13:19,578: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-05-09 21:13:19,582: INFO: common: yaml file: params.yaml loaded successfully]
artifacts/data_ingestion/test
[2025-05-09 21:13:19,585: INFO: common: created directory at: artifacts]
Found 36 images belonging to 2 classes.
3/3 [==============================] - 6s 1s/step - loss: 3.7592 - accuracy: 0.6389
[2025-05-09 21:13:29,361: INFO: common: json file saved at: scores.json]


2025/05/09 21:13:31 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2025-05-09 21:13:47,689: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 54). These functions will not be directly callable after loading.]
INFO:tensorflow:Assets written to: C:\Users\Admin\AppData\Local\Temp\tmppernng8t\model\data\model\assets
[2025-05-09 21:13:53,487: INFO: builder_impl: Assets written to: C:\Users\Admin\AppData\Local\Temp\tmppernng8t\model\data\model\assets]


Registered model 'ResNet50Model' already exists. Creating a new version of this model...
2025/05/09 21:14:59 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: ResNet50Model, version 5
Created version '5' of model 'ResNet50Model'.
